# <center> US pollution analysis

### Introduction

The US Pollution Data project leverages a comprehensive dataset covering air pollution measurements across the United States — consisting of over 1.4 million observations and around 28 variables that record concentrations of major pollutants such as nitrogen dioxide (NO₂), sulfur dioxide (SO₂), carbon monoxide (CO), and ozone (O₃). 


This dataset spans several years and states, allowing detailed spatio-temporal analysis of pollutant levels, seasonal trends, and geographic variation. The primary objective of this project is to transform this raw data into clean, analysis-ready formats (e.g., Parquet), conduct exploratory data analysis (EDA) to uncover patterns and insights, and — where possible — apply machine learning techniques to forecast pollutant concentrations, classify air‑quality levels, or identify key factors driving pollution.

Given the public health importance of air quality, this project has the potential not only to improve our understanding of pollution trends in the U.S., but also to inform policy, raise awareness, or support predictive systems that warn populations about deteriorating air conditions.

The goal of this project is to:

* Clean and transform the raw data (e.g., into Parquet)
* Perform EDA to uncover trends, seasonal patterns, and geographic variation
* Apply ML techniques to forecast pollutant levels, classify air quality, and analyze feature importance

By improving our understanding of air quality trends, this project supports public health insights and data-driven policy decisions.

Dataset content:
* State Code: Numeric code representing the U.S. state
* County Code:	Numeric code for the county within the state
* Site Num:	Identifier for the air monitoring site
* Address:	Street address of the monitoring station
* State:	Full name of the U.S. state
* County:	Name of the county
* City:	City where the measurement site is located
* Date Local:	Date of the observation (YYYY-MM-DD)
* NO2 Units:	Units used for nitrogen dioxide measurements
* NO2 Mean:	Daily average NO₂ concentration
* NO2 1st Max Value:	Highest NO₂ value recorded that day
* NO2 1st Max Hour:	Hour when the highest NO₂ was recorded
* NO2 AQI:	Air Quality Index for NO₂ on that day
* O3 Units:	Units used for ozone measurements
* O3 Mean:	Daily average ozone concentration
* O3 1st Max Value:	Highest O₃ value recorded that day
* O3 1st Max Hour:	Hour when the highest O₃ was recorded
* O3 AQI:	Air Quality Index for O₃ on that day
* SO2 Units:	Units used for sulfur dioxide measurements
* SO2 Mean:	Daily average SO₂ concentration
* SO2 1st Max Value:	Highest SO₂ value recorded that day
* SO2 1st Max Hour:	Hour when the highest SO₂ was recorded
* SO2 AQI:	Air Quality Index for SO₂ on that day
* CO Units:	Units used for carbon monoxide measurements
* CO Mean:	Daily average CO concentration
* CO 1st Max Value:	Highest CO value recorded that day
* CO 1st Max Hour:	Hour when the highest CO was recorded
* CO AQI:	Air Quality Index for CO on that day

---

### 1. EDA and Initial data visualisation

First, all necessary libraries are imported

In [ ]:
import os                          #import os for operating system interactions
import pandas as pd                 #import Pandas for data manipulation
import numpy as np                  #import Numpy for numerical operations
import matplotlib.pyplot as plt     #import Matplotlib for data visualization
import seaborn as sns               #import Seaborn for statistical data visualization
from plotly.subplots import make_subplots  #import Plotly subplots for creating complex figures
import plotly.express as px         #import Plotly Express for interactive visualizations
import plotly.graph_objects as go   #import Plotly Graph Objects for detailed figure customization

In [ ]:
sns.set(style="whitegrid")                  # Set Seaborn style for plots
plt.rcParams["figure.figsize"] = (10,6)     # Set default figure size for Matplotlib plots

#### 1.1. ETL and EDA


In this section EDA, including data load and cleaning, is performed. As a first step, data set is loaded into DataFrame

In [ ]:
df = pd.read_parquet("../data/raw/pollution_dataset.parquet", engine="pyarrow")
df.head()

In [ ]:
df. drop(columns=['Unnamed: 0'], inplace=True)  # Drop unnecessary column
df.head()

In the following subsection initial data set inspection is performed. Here the shape and Info of DataFrame are shown

In [ ]:
print(df.shape)                     # Print the shape of the DataFrame           
print(df.info())                    # Print concise summary of the DataFrame            
print(df.dtypes)                    # Print data types of each column

In [ ]:
df.dtypes.value_counts()         # Count occurrences of each data type

As it shown above dataset consists of 1746661 entries and 28 columns. Also dataset contains 10 float columns, 9 integer and 9 categorical columns. 

In the next steps DataFrame is checked for any incosistencies(dublicates, missing value and etc.)

In [ ]:
df.isnull().sum()           # Check for missing values in each column

The dataset misses 872907 and 873323 values in SO2 AQI and CO AQI columns respectively. This is a big amount of missing data to just remove lines. Instead, these missing values can be calculated, as far as other columns have no missing points.

Air Quality Index (AQI) is calculated by converting measured pollutant concentrations (e.g., SO₂, CO, NO₂, O₃, PM₂.₅, PM₁₀) into a standardized scale (usually 0–500) using breakpoints.
Core AQI Formula

Each pollutant gets its own AQI number. The final AQI for the city/location is the highest of all pollutants.

Each government sets concentration ranges for each pollutant.
Example (US EPA standard):
SO₂ 1-hour breakpoints (ppb)
AQI Range	                SO₂ (ppb)
0–50 (Good)	                0–35
51–100 (Moderate)	        36–75
101–150 (Unhealthy SG)	    76–185
151–200 (Unhealthy)	        186–304
201–300 (Very Unhealthy)    305–604

CO 8-hour breakpoints (ppm)
AQI Range	CO (ppm)
0–50	    0.0–4.4
51–100	    4.5–9.4
101–150	    9.5–12.4
151–200	    12.5–15.4
201–300	    15.5–30.4

Before calculating AQIs SO2 and CO, let's confirm that dedicated cilumns do not contain negeative values (which is physically impossible)

In [ ]:
neg_so2 = df[df["SO2 1st Max Value"] < 0][["SO2 Units", "SO2 Mean", "SO2 1st Max Value"]]
neg_co = df[df["CO Mean"] < 0][["CO Units", "CO Mean", "CO 1st Max Value"]]

print("Negative SO₂ values:")
print(neg_so2.head())

print("\nNegative CO values:")
print(neg_co.head())


In [ ]:
print(neg_co.value_counts().sum())
print(neg_so2.value_counts().sum())

There are 1064 and 8286 negative values in CO Mean and SO2 1st Max Value columns, respectively. In this case, removing these values from the dataset is the simplest, cleanest and safest approach.

Negative pollution values are invalid. SO₂ and CO cannot be negative in reality. These values come from:
* sensor malfunction
* data ingestion error
* interpolation issues

Removing them does not lose valid information.

Only a tiny fraction of data is affected. There are 1,746,661 total rows. Problematic rows:
* SO₂ negatives: 8,286
* CO negatives: 1,064
* Combined: < 0.5% of data

Removing them has zero statistical impact on AQI analysis.

Deleting invalid rows

In [ ]:
df = df[(df["SO2 1st Max Value"] >= 0) & (df["CO Mean"] >= 0)]
print(df.shape)

Below SO2 AQI column calculated based on given values in dedicated SO2 columns. 

In [ ]:

def calculate_so2_aqi(C):
    """
    Calculate SO2 AQI using expanded breakpoint intervals (Option C).
    This avoids NA values from strict EPA bins.
    
    C : float 
        1-hour SO2 concentration in ppb
    """

    if pd.isna(C):
        return np.nan

    # ===== Expanded breakpoints ensuring continuous coverage =====
    if 0 <= C <= 35.999:
        Clow, Chigh = 0, 35
        Ilow, Ihigh = 0, 50

    elif 36 <= C <= 75.999:
        Clow, Chigh = 36, 75
        Ilow, Ihigh = 51, 100

    elif 76 <= C <= 185.999:
        Clow, Chigh = 76, 185
        Ilow, Ihigh = 101, 150

    elif 186 <= C <= 304.999:
        Clow, Chigh = 186, 304
        Ilow, Ihigh = 151, 200

    elif 305 <= C <= 604.999:
        Clow, Chigh = 305, 604
        Ilow, Ihigh = 201, 300

    else:
        # Out of range but we extend for safety
        return np.nan

    # ===== AQI Formula =====
    aqi = ((Ihigh - Ilow) / (Chigh - Clow)) * (C - Clow) + Ilow
    return round(aqi, 1)


In [ ]:
# Function to add SO2 AQI column to DataFrame
def add_so2_aqi_column(df, col_name="SO2 1st Max Value"):
    """
    df : pandas dataframe  
    col_name : column containing SO2 1-hour max in ppb
    """
    df = df.copy()
    df["SO2 AQI"] = df[col_name].apply(calculate_so2_aqi)
    return df

In [ ]:
df = add_so2_aqi_column(df, "SO2 1st Max Value")
df["SO2 AQI"].isna().sum()

And check column  and values

In [ ]:
print(df[["SO2 1st Max Value", "SO2 AQI"]].head())

And CO AQI calculation

In [ ]:
import numpy as np
import pandas as pd

def calculate_co_aqi(C):
    """
    Calculate CO AQI using expanded breakpoint intervals (Option C).
    This ensures continuous coverage with NO missing AQI values.

    C : float 
        8-hour CO concentration in ppm
    """

    if pd.isna(C):
        return np.nan

    # ===== Expanded breakpoints =====
    if 0.0 <= C <= 4.499:
        Clow, Chigh = 0.0, 4.4
        Ilow, Ihigh = 0, 50

    elif 4.5 <= C <= 9.499:
        Clow, Chigh = 4.5, 9.4
        Ilow, Ihigh = 51, 100

    elif 9.5 <= C <= 12.499:
        Clow, Chigh = 9.5, 12.4
        Ilow, Ihigh = 101, 150

    elif 12.5 <= C <= 15.499:
        Clow, Chigh = 12.5, 15.4
        Ilow, Ihigh = 151, 200

    elif 15.5 <= C <= 30.499:
        Clow, Chigh = 15.5, 30.4
        Ilow, Ihigh = 201, 300

    else:
        return np.nan  # extremely high or wrong units

    # ===== AQI Formula =====
    aqi = ((Ihigh - Ilow) / (Chigh - Clow)) * (C - Clow) + Ilow
    return round(aqi, 1)


In [ ]:
def add_co_aqi_column(df, col="CO Mean"):
    df = df.copy()
    df["CO AQI"] = df[col].apply(calculate_co_aqi)
    return df

In [ ]:
df = add_co_aqi_column(df, "CO Mean")
df["CO AQI"].isna().sum()

Verifying imputations

In [ ]:
df.isnull().sum()           # Check for missing values in each column

And searching for negative valeus in other numeric columns:

In [ ]:
def validate_no_negative_values(df, auto_fix=False, stop_on_error=False):
    """
    Validate that no negative values exist in any numeric column.
    
    auto_fix: If True → removes all rows containing negative values.
    stop_on_error: If True → raises an exception if negatives exist.
    """

    numeric_cols = df.select_dtypes(include=[np.number]).columns

    negatives_report = {}

    for col in numeric_cols:
        neg_mask = df[col] < 0
        neg_count = neg_mask.sum()

        if neg_count > 0:
            negatives_report[col] = {
                "count": int(neg_count),
                "sample": df.loc[neg_mask].head()
            }

    # If clean → report success
    if len(negatives_report) == 0:
        print("✅ No negative values found in ANY numeric column.")
        return df

    # If negatives exist → print detailed report
    print("❌ Negative values found!\n")

    for col, info in negatives_report.items():
        print(f"Column: {col}")
        print(f"  → Negative count: {info['count']}")
        print(f"  → Sample rows with negatives:")
        print(info["sample"])
        print("-" * 40)

    # Auto-fix option: remove rows
    if auto_fix:
        print("🧹 Auto-fix: Removing rows containing any negative values...")
        df = df[(df[numeric_cols] >= 0).all(axis=1)]
        print("✔ Negative rows removed.")
        return df

    # Stop execution option
    if stop_on_error:
        raise ValueError("Dataset contains negative values! See report above.")

    return df


In [ ]:
validate_no_negative_values(df)
df = validate_no_negative_values(df, auto_fix=True)
df = validate_no_negative_values(df, stop_on_error=True)


828 negative valeus were found in NO2 mean columns. These rows were automatically deleted.

In [ ]:
print(df.shape)                     # Print the shape of the DataFrame  

Checking for TRUE duplicates.

In [ ]:
def find_full_pollutant_duplicates(df):
    """
    Find duplicates using:
    - Date Local
    - Address
    - All pollutant mean / max / hour values
    - All AQI values
    
    Shows duplicates but DOES NOT remove them.
    """

    # Identify pollutant measurement columns dynamically
    pollutant_cols = [
        col for col in df.columns 
        if any(p in col for p in ["NO2", "O3", "SO2", "CO"])
    ]

    # Identify AQI columns
    aqi_cols = [col for col in df.columns if col.endswith("AQI")]

    # Build final set of columns for duplicate detection
    key_columns = ["Date Local", "Address"] + pollutant_cols + aqi_cols

    print("🔍 Checking duplicates using ALL pollutant columns:")
    print(key_columns, "\n")

    # Find duplicates (count both first and later occurrences)
    dup_mask = df.duplicated(subset=key_columns, keep=False)
    duplicates = df.loc[dup_mask].sort_values(by=key_columns)

    print(f"📌 Total FULL pollutant duplicates found: {len(duplicates)}\n")

    if len(duplicates) == 0:
        print("✅ No duplicates found.")
        return duplicates

    print("📄 Sample of duplicate rows (first 30):")
    display(duplicates.head(30))

    print("\n📊 Duplicate groups summary:")
    group_counts = (
        duplicates.groupby(["Date Local", "Address"])
        .size()
        .reset_index(name="Count")
        .sort_values("Count", ascending=False)
    )
    display(group_counts.head(20))

    return duplicates


In [ ]:
duplicates = find_full_pollutant_duplicates(df)

Remove only TRUE Duplicates, perfectly matching.

This function:
* Removes only exact duplicates
*  Keeps the first occurrence
* Shows how many were removed
* Shows which Date/Address pairs had duplicates
* Returns a cleaned dataframe

In [ ]:
def remove_full_pollutant_duplicates(df):
    """
    Remove ONLY true duplicates based on:
    - Date Local
    - Address
    - All pollutant measurement columns (Mean, Max, Hour, Units)
    - All AQI columns

    Keeps the FIRST occurrence.
    Returns cleaned dataframe + summary report.
    """

    df_clean = df.copy()

    # Identify pollutant columns dynamically
    pollutant_cols = [
        col for col in df.columns
        if any(p in col for p in ["NO2", "O3", "SO2", "CO"])
    ]

    # Identify AQI columns
    aqi_cols = [col for col in df.columns if col.endswith("AQI")]

    key_columns = ["Date Local", "Address"] + pollutant_cols + aqi_cols

    print("🧹 Removing true duplicates based on columns:")
    print(key_columns, "\n")

    before = len(df_clean)

    # Remove duplicates (keep first occurrence)
    df_clean = df_clean.drop_duplicates(subset=key_columns, keep="first")

    after = len(df_clean)
    removed = before - after

    print(f"📉 Total rows BEFORE: {before}")
    print(f"📈 Total rows AFTER:  {after}")
    print(f"🗑️ Removed duplicates: {removed}\n")

    # Show top duplicate groups (optional)
    if removed > 0:
        print("📊 Duplicate groups (Date + Address) impacted:")
        dup_groups = (
            df[df.duplicated(subset=key_columns, keep=False)]
            .groupby(["Date Local", "Address"])
            .size()
            .reset_index(name="Count")
            .sort_values("Count", ascending=False)
        )
        display(dup_groups.head(20))
    else:
        print("✅ No duplicates were removed. Dataset was already clean.")

    return df_clean


In [ ]:
clean_df = remove_full_pollutant_duplicates(df)


Conflicting duplicates identification:
* Detect measurement inconsistencies
* Identify stations that report multiple measurements at same time
* Flag data quality problems
* Decide whether to:
 * average conflicts
 * drop the worst sensors
 * keep the max value (common for AQI rules)

Resolve Conflicting Duplicates. Keep the Maximum Values (EPA-style AQI logic)

EPA rules for AQI calculations already require maxima for 1-hour & 8-hour values.
So to remain consistent, we choose the maximum values within each conflict group.

* Best for AQI
* Prevents underestimation
* Officially aligned with U.S. EPA methodology

It ensures:
* proper pollutant selection
* AQI is never underestimated
* dataset integrity for environmental analysis

In [ ]:
[x for x in df.columns if df.columns.tolist().count(x) > 1]


In [ ]:

def detect_and_resolve_conflicts(df):
    """
    Detect and resolve conflicting duplicates based on:
    - Same Date Local + Address
    - Pollutant or AQI values differ
    Resolution rule: keep MAX values (EPA-style).

    Returns:
        df_cleaned : dataframe with resolved conflicts
        conflicts  : dataframe containing original conflicting rows
    """

    df_copy = df.copy()

    # -----------------------------
    # Identify pollutant + AQI columns
    # -----------------------------
    pollutant_cols = [c for c in df_copy.columns if any(p in c for p in ["NO2", "O3", "SO2", "CO"])]
    aqi_cols = [c for c in df_copy.columns if c.endswith("AQI")]
    group_cols = pollutant_cols + aqi_cols

    key_cols = ["Date Local", "Address"]

    print("🔍 Checking columns:")
    print("  Pollutant cols:", pollutant_cols)
    print("  AQI cols:", aqi_cols)
    print("  Keys:", key_cols, "\n")

    # -----------------------------
    # STEP 1 — Detect conflicting groups
    # -----------------------------
    grouped = (
        df_copy.groupby(key_cols)[group_cols]
        .nunique(dropna=False)
        .reset_index()
    )
    grouped["conflict"] = grouped[group_cols].max(axis=1) > 1

    conflict_keys = grouped[grouped["conflict"]][key_cols]

    if len(conflict_keys) == 0:
        print("✅ No conflicting duplicates found.")
        return df_copy, pd.DataFrame()

    print(f"⚠️ Total conflicting groups found: {len(conflict_keys)}")

    # Extract full conflicting rows
    conflicts = df_copy.merge(conflict_keys, on=key_cols, how="inner")
    print(f"⚠️ Total conflicting rows: {len(conflicts)}")

    # -----------------------------
    # STEP 2 — Resolve conflicts using MAX for pollutant/AQI cols
    # -----------------------------
    resolved_conflicts = (
        conflicts.groupby(key_cols)[group_cols]
        .max()
        .reset_index()
    )

    # -----------------------------
    # STEP 3 — Merge resolved rows back into dataset
    # -----------------------------
    # Remove old conflicting rows
    non_conflicts = df_copy.merge(
        conflict_keys, on=key_cols, how="left", indicator=True
    )
    non_conflicts = non_conflicts[non_conflicts["_merge"] == "left_only"]
    non_conflicts = non_conflicts.drop(columns=["_merge"])

    # Align column order
    resolved_conflicts = resolved_conflicts.reindex(columns=df_copy.columns, fill_value=np.nan)

    # Final dataset
    df_cleaned = pd.concat([non_conflicts, resolved_conflicts], ignore_index=True)

    print("\n🛠 Conflict Resolution Summary:")
    print(f"  Conflicting groups : {len(conflict_keys)}")
    print(f"  Rows removed       : {len(conflicts)}")
    print(f"  Rows added         : {len(resolved_conflicts)}")
    print(f"  Final dataset size : {len(df_cleaned)}")

    return df_cleaned, conflicts


In [ ]:
def fix_duplicate_columns(df):
    """
    Ensures column names are unique by automatically renaming duplicates.
    """
    new_cols = []
    seen = {}

    for col in df.columns:
        if col not in seen:
            new_cols.append(col)
            seen[col] = 1
        else:
            new_name = f"{col}_{seen[col]}"
            while new_name in seen:
                seen[col] += 1
                new_name = f"{col}_{seen[col]}"
            new_cols.append(new_name)
            seen[col] += 1

    df.columns = new_cols
    return df


In [ ]:
df = fix_duplicate_columns(df)

In [ ]:
df_cleaned, conflicts = detect_and_resolve_conflicts(df)


In [ ]:
conflicts.head(20)


In [ ]:
df_cleaned.head()
